In [69]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import sampler

import torchvision.transforms as T

In [67]:
USE_GPU = True
dtype = torch.float32

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [54]:
# Read in POC data
small_data = pd.read_pickle("/Users/josephjamison/Downloads/CS231n_temp/4k_POC_small_data.pkl")
small_labels = pd.read_pickle("4k_POC_small_labels.pkl")
large_data = pd.read_pickle("/Users/josephjamison/Downloads/CS231n_temp/4k_POC_large_data.pkl")
large_labels = pd.read_pickle("4k_POC_large_labels.pkl")

In [55]:
# Re-size POC data and store as np arrays
small_dim = 128
large_dim = 256
small_data = small_data.to_numpy().reshape(-1, 3, small_dim, small_dim)
small_labels = small_labels.to_numpy()
large_data = large_data.to_numpy().reshape(-1, 3, large_dim, large_dim)
large_labels = large_labels.to_numpy()

print(small_data.shape)
print(small_labels.shape)
print(large_data.shape)
print(large_labels.shape)

(4000, 3, 128, 128)
(4000, 1)
(4000, 3, 256, 256)
(4000, 1)


In [56]:
label_dict = {}
counter = 0
for i in np.unique(small_labels):
    label_dict[i] = counter
    counter += 1

print(label_dict)

{'airport_terminal': 0, 'bathroom': 1, 'dining_room': 2, 'highway': 3, 'skyscraper': 4}


In [57]:
# Convert labels from strings to ints
for i,j in label_dict.items():
    small_labels[small_labels==i] = j
    large_labels[large_labels==i] = j

small_labels = small_labels.astype(int)
large_labels = large_labels.astype(int)

In [62]:
small_X_train, small_X_val, small_y_train, small_y_val = train_test_split(small_data, small_labels, test_size=0.1, random_state=42)
large_X_train, large_X_val, large_y_train, large_y_val = train_test_split(large_data, large_labels, test_size=0.1, random_state=42)

In [63]:
# Calculate channel mean and std for transformation of dataset
small_mean = np.mean(small_X_train, axis = (0, 2, 3))
small_std = np.std(small_X_train, axis = (0, 2, 3))
large_mean = np.mean(large_X_train, axis = (0, 2, 3))
large_std = np.std(large_X_train, axis = (0, 2, 3))

In [64]:
small_transform = T.Normalize(small_mean, small_std)
large_transform = T.Normalize(large_mean, large_std)

small_X_train = small_transform.forward(torch.Tensor(small_X_train))
small_X_val = small_transform.forward(torch.Tensor(small_X_val))
small_y_train = torch.Tensor(small_y_train)
small_y_val = torch.Tensor(small_y_val)

large_X_train = large_transform.forward(torch.Tensor(large_X_train))
large_X_val = large_transform.forward(torch.Tensor(large_X_val))
large_y_train = torch.Tensor(large_y_train)
large_y_val = torch.Tensor(large_y_val)

In [65]:
# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches.

SMALL_DATA = True

# cifar10_train = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
#                              transform=transform)
if SMALL_DATA:
    train_dataset = TensorDataset(small_X_train, small_y_train)
    val_dataset = TensorDataset(small_X_val, small_y_val)
else:
    train_dataset = TensorDataset(large_X_train, large_y_train)
    val_dataset = TensorDataset(large_X_val, large_y_val)

loader_train = DataLoader(train_dataset, batch_size=64)

# cifar10_val = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
#                            transform=transform)
loader_val = DataLoader(val_dataset, batch_size=64)

# cifar10_test = dset.CIFAR10('./cs231n/datasets', train=False, download=True, 
#                             transform=transform)
# loader_test = DataLoader(cifar10_test, batch_size=64)

In [66]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image
    
# We need to wrap `flatten` function in a module in order to stack it
# in nn.Sequential
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

In [68]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        
    return num_correct, num_samples

In [70]:
def train(model, optimizer, epochs=1):    
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

        num_correct, num_samples = check_accuracy(loader_val, model)
        epoch_acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f) after epoch %d' % (num_correct, num_samples, 100 * epoch_acc, e+1))

    return epoch_acc

In [71]:
if SMALL_DATA:
    input_h = 128
else:
    input_h = 256
        
channel_1 = 16
channel_2 = 32
channel_3 = 16
channel_4 = 32

# First pool layer
kernel_size_1 = 2
h_out_pool_1 = (input_h - (kernel_size_1 - 1)-1) / kernel_size_1 + 1

# Second pool layer
kernel_size_2 = 4
h_out_pool_2 = (h_out_pool_1 - (kernel_size_2 - 1)-1) / kernel_size_2 + 1

channel_out = int(channel_4 * h_out_pool_2 * h_out_pool_2) # flattened output size for affine

EPOCHS = 2
learning_rate = 1e-3

model = nn.Sequential(
        # Layer 1: Conv - batchnorm - relu - conv - batchnorm - relu - maxpool
        nn.Conv2d(in_channels= 3, out_channels= channel_1, kernel_size= (7,7), padding=3),
        nn.BatchNorm2d(channel_1),
        nn.ReLU(),
        nn.Conv2d(in_channels= channel_1, out_channels= channel_2, kernel_size= (5,5), padding=2),
        nn.BatchNorm2d(channel_2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size= kernel_size_1),
        # Layer 2: Conv - batchnorm - relu - conv - relu - maxpool
        nn.Conv2d(in_channels= channel_2, out_channels= channel_3, kernel_size= (3,3), padding=1),
        nn.BatchNorm2d(channel_3),
        nn.ReLU(),
        nn.Conv2d(in_channels= channel_3, out_channels= channel_4, kernel_size= (1,1), padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size= kernel_size_2),
        # Output: Affine
        Flatten(),
        nn.Linear(channel_out, 10))

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

acc = train(model, optimizer, epochs = EPOCHS)

RuntimeError: 0D or 1D target tensor expected, multi-target not supported